## OPTIMIZING MODEL PARAMETERS
## 优化模型参数

现在我们有了模型和数据，是时候通过优化数据上的参数来训练、验证和测试我们的模型了。训练模型是一个迭代过程；在每次迭代（称为epoch）中，模型对输出进行猜测，计算猜测中的误差（损失），收集误差相对于其参数的导数（如我们在上一节中所见），并优化这些参数使用梯度下降

### Prerequisite Code
### 先决条件代码

加载Datasets，DataLoaders和构建的模型

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### Hyperparameters
### 超参数

超参数是可调整的参数，可让您控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（阅读有关超参数调整的更多信息）

为训练定义了以下超参数：
- Number of Epochs - 迭代数据集的次数
- Batch Size - 参数更新前通过网络传播的数据样本数
- 学习率- 在每个批次/时期更新模型参数的程度。较小的值会产生较慢的学习速度，而较大的值可能会导致训练期间出现不可预测的行为。

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop
### 优化循环

一旦设置了超参数，就可以使用优化循环来训练和优化模型。优化循环的每次迭代称为epoch。

每个时期包括两个主要部分：
- 训练循环- 迭代训练数据集并尝试收敛到最佳参数
- 验证/测试循环- 迭代测试数据集以检查模型性能是否正在改善

### Loss Function
### 损失函数

当呈现一些训练数据时，我们未经训练的网络可能不会给出正确的答案。损失函数衡量得到的结果与目标值的相异程度，是我们在训练时要最小化的损失函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括用于回归任务的nn.MSELoss（均方误差）和 用于分类的nn.NLLLoss（负对数似然）。 nn.CrossEntropyLoss结合nn.LogSoftmax和nn.NLLLoss。

我们将模型的输出 logits 传递给nn.CrossEntropyLoss，这将对 logits 进行归一化并计算预测误差。

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer
### 优化器

优化是在每个训练步骤中调整模型参数以减少模型误差的过程。优化算法定义了如何执行这个过程（在这个例子中，我们使用随机梯度下降）。所有优化逻辑都封装在optimizer对象中。在这里，我们使用 SGD 优化器；此外，PyTorch 中有许多不同的优化器 可用，例如 ADAM 和 RMSProp，它们可以更好地用于不同类型的模型和数据。

我们通过记录模型需要训练的参数并传入学习率超参数来初始化优化器

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分三个步骤进行：
- 调用optimizer.zero_grad()以重置模型参数的梯度。默认情况下渐变加起来；为了防止重复计算，我们在每次迭代时明确地将它们归零。
- 通过调用来反向传播预测损失loss.backward()。PyTorch 存储每个参数的损失梯度。
- 一旦我们有了我们的梯度，我们调用optimizer.step()通过在反向传递中收集的梯度来调整参数。

### Full Implementation
### 完整执行

定义train_loop循环优化代码，并test_loop根据我们的测试数据评估模型的性能

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

初始化损失函数和优化器，并将其传递给train_loop和test_loop。随意增加 epoch 的数量来跟踪模型的改进性能。

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.293444  [    0/60000]
loss: 2.290363  [ 6400/60000]
loss: 2.274277  [12800/60000]
loss: 2.278600  [19200/60000]
loss: 2.246149  [25600/60000]
loss: 2.220686  [32000/60000]
loss: 2.229470  [38400/60000]
loss: 2.190457  [44800/60000]
loss: 2.189467  [51200/60000]
loss: 2.168111  [57600/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.154398 

Epoch 2
-------------------------------
loss: 2.156389  [    0/60000]
loss: 2.152673  [ 6400/60000]
loss: 2.094094  [12800/60000]
loss: 2.118271  [19200/60000]
loss: 2.055144  [25600/60000]
loss: 2.001046  [32000/60000]
loss: 2.033792  [38400/60000]
loss: 1.947204  [44800/60000]
loss: 1.958464  [51200/60000]
loss: 1.902651  [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 1.884687 

Epoch 3
-------------------------------
loss: 1.911248  [    0/60000]
loss: 1.890589  [ 6400/60000]
loss: 1.768520  [12800/60000]
loss: 1.818163  [19200/60000]
loss: 1.700204  [25600/60000]
loss: 1.659029  [32000/600